In [6]:
from typing import Dict,Tuple,List,Union
import pandas as pd
import numpy as np
from scr.core import HMA
from scr.load_excel_data import query_data,query_sw_classify,query_stock_index_classify,load_excel

# 价量共振

## 价量共振指标的择时系统构建

1. 成交量使用的移动平均线为 AMA，收盘价使用的移动平均线为 BMA(移动平均线长度为L);
2. 价能=$\frac{BMA_{today}}{BMA_{Today-N}}$，当日的BMA除以前N个交易日的BMA;
3. 量能=$\frac{AMA_{5}}{AMA_{Long}}$，5 日的AMA除以长期的AMA;
4. 价量共振指标=价能×量能，价能乘以量能
5. 当价量共振指标大于 Threshold 则做多，否则平仓

## 市场划分条件下的择时系统

1. 当5日均线高于90日均线，市场划分为多头市场；当5日均线小于90日均线，市场划分为空头市场;
2. 当前为多头市场下，若价量共振指标大于Threshold1则做多，否则以Threshold1平仓。当前为空头市场下， 若价量共振指标大于Threshold2则做多，否则以Threshold2平仓;

**参数**

AMA长期移动平均线长度为 100，BMA移动平均线长度为 50，N=3，Threshold1=1.125，Threshold2=1.275

In [7]:
hs300:pd.DataFrame = query_data('000300.SH','2010-01-01','2022-10-11',fields=['close','volume'],method='index')
hs300.set_index('trade_date',inplace=True)

In [12]:
N:int = 3
THRESHOLD1:float = 1.125
THRESHOLD2:float = 1.275

# 价能指标

BMA:pd.Series = HMA(hs300['close'],50)

price_mom:pd.Series = BMA / BMA.shift(N)

# 量能指标
vol_mom:pd.Series = HMA(hs300['volume'],5) / HMA(hs300['volume'],100)

price_vol:pd.Series = price_mom * vol_mom

In [16]:
filter_shake:pd.Series = (hs300['close'].rolling(5).mean() > hs300['close'].rolling(90).mean())

In [18]:
threshold_ser:pd.Series = filter_shake.apply(lambda x:np.where(x,THRESHOLD1,THRESHOLD2))

In [19]:
price_vol

trade_date
2010-01-04         NaN
2010-01-05         NaN
2010-01-06         NaN
2010-01-07         NaN
2010-01-08         NaN
                ...   
2022-09-28    0.943916
2022-09-29    0.898258
2022-09-30    0.859400
2022-10-10    1.008765
2022-10-11    1.041870
Length: 3101, dtype: float64

In [20]:
threshold_ser

trade_date
2010-01-04    1.275
2010-01-05    1.275
2010-01-06    1.275
2010-01-07    1.275
2010-01-08    1.275
              ...  
2022-09-28    1.275
2022-09-29    1.275
2022-09-30    1.275
2022-10-10    1.275
2022-10-11    1.275
Name: close, Length: 3101, dtype: float64